In [ ]:

def similar(a, b):
    
    return SequenceMatcher(None, str(a), str(b)).ratio()

In [ ]:

def key_in_search(driver, input_xpath, field_value):
    print('Searching for: {}'.format(field_value))
    input_tag = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, input_xpath))
        )
    input_tag.click()
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).perform()
    input_tag.send_keys(field_value)
    ActionChains(driver).key_down(Keys.ENTER).perform()

In [ ]:

def unobscure_element(driver, match_tuple):
    tag_name = match_tuple[0]
    class_str = '.'.join([cn for cn in match_tuple[1].split(' ') if cn != 'hidden'])
    try:
        overlay_css = '{}.{}'.format(tag_name, class_str)

        # Wait for overlay div to show up
        overlay_tag = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, overlay_css))
            )
        print('Hiding <{}'.format(overlay_tag.get_attribute('outerHTML').split('<')[1]))
        driver.execute_script("arguments[0].setAttribute('style','display:none;');", overlay_tag)

    except Exception as e:
        message = str(e).strip()
        print('Waiting for {} {} to show up before hiding it: {}'.format(class_str, tag_name, message))
        driver.refresh()

In [ ]:

def move_to(driver, xpath, verbose=True):
    if verbose:
        print('Moving to {}'.format(xpath))
    try:
        web_element = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.XPATH, xpath))
            )
        ActionChains(driver).move_to_element(web_element).perform()
    except Exception as e:
        message = str(e).strip()
        if verbose:
            print('Waiting for the web element to be visible: {}'.format(message))
        if 'obscures' in message:
            match_obj = obscure_regex.search(message)
            while match_obj:
                unobscure_element(driver, match_obj)
                try:
                    web_element = WebDriverWait(driver, 5).until(
                        EC.visibility_of_element_located((By.XPATH, xpath))
                        )
                    ActionChains(driver).move_to_element(web_element).perform()
                    message = ''
                except Exception as e:
                    message = str(e).strip()
                    if verbose:
                        print('Waiting for the web element to be visible (unobscured): {}'.format(message))
                match_obj = obscure_regex.search(message)

In [ ]:

def click_web_element(driver, xpath):
    print('Clicking {}'.format(xpath))
    try:
        web_element = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.XPATH, xpath))
            )
        ActionChains(driver).move_to_element(web_element).perform()
        web_element.click()
    except Exception as e:
        message = str(e).strip()
        print('Waiting for the web element to be visible: {}'.format(message))
        if ('obscures' in message) or ('Other element would receive the click' in message):
            tag_list = obscure_regex.findall(message)
            while len(tag_list) == 2:
                unobscure_element(driver, tag_list[1])
                try:
                    web_element = WebDriverWait(driver, 5).until(
                        EC.visibility_of_element_located((By.XPATH, xpath))
                        )
                    ActionChains(driver).move_to_element(web_element).perform()
                    web_element.click()
                    message = ''
                except Exception as e:
                    message = str(e).strip()
                    print('Waiting for the web element to be visible (unobscured): {}'.format(message))
                tag_list = obscure_regex.findall(message)

In [ ]:

def get_element_contents(driver, xpath, verbose=True, scroll=True):
    if verbose:
        print('Getting text of {}'.format(xpath))
    results_str = ''
    try:
        if scroll:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        web_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, xpath))
            )
        if scroll:
            driver.execute_script('arguments[0].scrollIntoView(true)', web_element)
        results_str = web_element.text.strip()
    except Exception as e:
        message = str(e).strip()
        if verbose:
            print('Waiting for the web element to be located: {}'.format(message))
        if ('obscures' in message) or ('Other element would receive the click' in message):
            tag_list = obscure_regex.findall(message)
            while len(tag_list) == 2:
                unobscure_element(driver, tag_list[1])
                try:
                    if scroll:
                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                    web_element = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, xpath))
                        )
                    if scroll:
                        driver.execute_script('arguments[0].scrollIntoView(true)', web_element)
                    results_str = web_element.text.strip()
                    message = ''
                except Exception as e:
                    message = str(e).strip()
                    if verbose:
                        print('Waiting for the web element to be located (unobscured): {}'.format(message))
                tag_list = obscure_regex.findall(message)
    
    return results_str

In [ ]:

def create_keywords_file(test_name='test_one', keyword_list=['Bedsheets', 'Clocks', 'Padlocks']):
    wb = Workbook()
    xlsx_dir = os.path.join('../saves', 'xlsx')
    os.makedirs(name=xlsx_dir, exist_ok=True)

    # grab the active worksheet
    ws = wb.active

    # Data can be assigned directly to cells
    ws['A1'] = 'Search Term'
    ws['A1'].font = Font(bold=True)
    column_width = 11.14
    ws.column_dimensions['A'].width = column_width
    for i, keyword_str in enumerate(keyword_list):
        ws['A{}'.format(i+2)] = keyword_str

    # Save the file
    file_path = os.path.join(xlsx_dir, '{}.xlsx'.format('_'.join([test_name, 'keywords'])))
    print('Saving to {}'.format(os.path.abspath(file_path)))
    wb.save(file_path)
    
    return file_path

In [ ]:

def get_contains_list(div_class_list):
    
    return ["contains(@class, '{}')".format(c) for c in div_class_list]

In [ ]:

def get_div(div_class_str):
    
    return 'div[{}]'.format(' and '.join(get_contains_list(div_class_str.split(' '))))

In [ ]:

def test_diaper():
    driver = get_driver('Chrome')
    driver_get_url(driver, url_str='http://www.firstcry.com/')
    time.sleep(20)
    
    # 3. On http://www.firstcry.com/, search for the items related to the below given product names.
    input_xpath = '/html/body/div[5]/div[2]/div[1]/div[1]/form/input'
    key_in_search(driver, input_xpath, 'Diaper')
    
    # Select the option to reload the results in descending order of the price.
    arrow_xpath = '/html/body/{}/div[2]/div/div[2]/div[1]/div[2]/div[1]/div/span[2]'.format(get_div('list_back'))
    click_web_element(driver, arrow_xpath)
    price_xpath = '/html/body/{}/div[2]/div/div[2]/div[1]/div[2]/div[1]/ul/li[4]/a'.format(get_div('list_back'))
    click_web_element(driver, price_xpath)
    
    # Write script for Google Chrome in PyTest/ WebDriver to test whether the first 8 products are
    # in descending order of the price. (15 points)
    price_regex = re.compile(r'</span>([.0-9]+)</a>')
    products_xpath = '/html/body/{}/div[2]/div/div[2]/div[3]/div[1]/div[3]'.format(get_div('list_back'))
    price_list = []
    for i in range(8):
        div_xpath = '{}/div[{}]/div/div[1]'.format(products_xpath, i+1)
        move_to(driver, div_xpath, verbose=False)
        div_tag = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, div_xpath))
            )
        match_obj = price_regex.search(div_tag.get_attribute('innerHTML'))
        if match_obj:
            price_list.append(float(match_obj.group(1)))
    
    driver.close()
    
    assert price_list == sorted(price_list)

In [ ]:

def test_clock():
    
    # 3. On http://www.firstcry.com/, search for the items related to the below given product names.
    input_xpath = '/html/body/div[5]/div[2]/div[1]/div[1]/form/input'
    key_in_search(driver, input_xpath, 'Clock')
    
    # Select the option to reload the results in descending order of the price.
    arrow_xpath = '/html/body/{}/div[2]/div/div[2]/div[1]/div[2]/div[1]/div/span[2]'.format(get_div('list_back'))
    click_web_element(driver, arrow_xpath)
    price_xpath = '/html/body/{}/div[2]/div/div[2]/div[1]/div[2]/div[1]/ul/li[4]/a'.format(get_div('list_back'))
    click_web_element(driver, price_xpath)
    
    # Write script for Google Chrome in PyTest/ WebDriver to test whether the first 8 products are
    # in descending order of the price. (15 points)
    price_regex = re.compile(r'</span>([.0-9]+)</a>')
    products_xpath = '/html/body/{}/div[2]/div/div[2]/div[3]/div[1]/div[3]'.format(get_div('list_back'))
    price_list = []
    for i in range(8):
        div_xpath = '{}/div[{}]/div/div[1]'.format(products_xpath, i+1)
        move_to(driver, div_xpath, verbose=False)
        div_tag = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, div_xpath))
            )
        match_obj = price_regex.search(div_tag.get_attribute('innerHTML'))
        if match_obj:
            price_list.append(float(match_obj.group(1)))
    
    driver.close()
    
    assert price_list == sorted(price_list)

In [ ]:

def test_shoe():
    driver = get_driver('Chrome')
    driver_get_url(driver, url_str='http://www.firstcry.com/')
    time.sleep(20)
    
    # 3. On http://www.firstcry.com/, search for the items related to the below given product names.
    input_xpath = '/html/body/div[5]/div[2]/div[1]/div[1]/form/input'
    key_in_search(driver, input_xpath, 'Shoe')
    
    # Select the option to reload the results in descending order of the price.
    arrow_xpath = '/html/body/{}/div[2]/div/div[2]/div[1]/div[2]/div[1]/div/span[2]'.format(get_div('list_back'))
    click_web_element(driver, arrow_xpath)
    price_xpath = '/html/body/{}/div[2]/div/div[2]/div[1]/div[2]/div[1]/ul/li[4]/a'.format(get_div('list_back'))
    click_web_element(driver, price_xpath)
    
    # Write script for Google Chrome in PyTest/ WebDriver to test whether the first 8 products are
    # in descending order of the price. (15 points)
    price_regex = re.compile(r'</span>([.0-9]+)</a>')
    products_xpath = '/html/body/{}/div[2]/div/div[2]/div[3]/div[1]/div[3]'.format(get_div('list_back'))
    price_list = []
    for i in range(8):
        div_xpath = '{}/div[{}]/div/div[1]'.format(products_xpath, i+1)
        move_to(driver, div_xpath, verbose=False)
        div_tag = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, div_xpath))
            )
        match_obj = price_regex.search(div_tag.get_attribute('innerHTML'))
        if match_obj:
            price_list.append(float(match_obj.group(1)))
    
    driver.close()
    
    assert price_list == sorted(price_list)

In [ ]:

# 3. On http://www.firstcry.com/, search for the items related to the below given product names.
# Select the option to reload the results in descending order of the price.
# Test1: Use product name “Diaper”,
# Test2: Use product name “Clock” and
# Test3: Use product name “Shoe”.
# These product names are to be stored in an Excel file, read the product names from this file.
# Write script for Google Chrome in PyTest/ WebDriver to test whether the first 8 products are
# in descending order of the price. (15 points)
wb = load_workbook(filename=create_keywords_file(test_name='test_three', keyword_list=['Diaper', 'Clock', 'Shoe']), read_only=True)

In [ ]:

# grab the active worksheet
ws = wb.active
cell_tuple_list = list(ws.iter_rows(min_row=2, max_row=None, min_col=1, max_col=1, values_only=True))

In [ ]:
for cell_tuple in cell_tuple_list:
    keyword_str = cell_tuple[0]
    print()
    print('#################################################################################')
    print('                             {}'.format(keyword_str))
    print('#################################################################################')
    print()
    if keyword_str == 'Diaper':
        test_diaper()
    if keyword_str == 'Clock':
        test_clock()
    if keyword_str == 'Shoe':
        test_shoe()